# Simple landscape exploration

Written by Simon M. Mudd at the University of Edinburgh. Updated 02/09/2021

## What is this notebook?

This is a template notebook for getting some data and then doing simple analyses:
* Getting a hillshade
* Getting a slope map
* Extracting channels
* Looking at the resulting data

## First steps: install some packages

If you are in our docker container you have the python packages you need. If you aren't, you need to run:

`!pip install lsdviztools lsdttparamselector`

Now impoort some packages for looking at the data:

In [ ]:
import lsdviztools.lsdbasemaptools as bmt
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio
import lsdviztools.lsdmapwrappers as lsdmw
import lsdttparamselector as ps
import pandas as pd
import geopandas as gpd
import cartopy as cp
import cartopy.crs as ccrs
import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np

## Now grab some data

First, we need to grab some data. We use a tool in `lsdviztools.lsdbasemaptools` called the `ot_scraper` (`ot` is for opentopography.org).

You can tell this what sort of data you want (most people will use the SRTM 30 metre data) and you also tell it the lower left and the upper right corners, using latitude and longitude. You can get these from goole earth by right clickjing on the map and selecting "what's here". In this example I use a place near Santa Barbara, California. 

In [ ]:
SB_DEM = bmt.ot_scraper(source = "SRTM30",longitude_W = -120.464655, longitude_E = -120.254214, latitude_S = 34.440538, latitude_N = 34.610770,prefix = "lg_conception")
SB_DEM.print_parameters()
SB_DEM.download_pythonic()

We need to convert the data because we are going to use some other software to analyse it.

In [ ]:
DataDirectory = "./"
RasterFile = "lg_conception_SRTM30.tif"
gio.convert4lsdtt(DataDirectory, RasterFile,minimum_elevation=0.01,resolution=30)

## Now we do some topographic analysis

We will extract some topogaphic metrics using `lsdtopotools`. You will need to install `lsdtopotools` for this. If you have `conda` install on your system you can install it with `!conda install -c conda-forge lsdtopotools`.

Here we assume `lsdtopotools` is installed. 

The `lsdtt_parameters` are the various parametes that you can use to run an analysis. We will discuss these later. For now, we will just follow this recipie. 

In [ ]:
lsdtt_parameters = {"write_hillshade" : "true",  
                    "surface_fitting_radius" : "60",
                    "print_slope" : "true"}
lsdtt_drive = lsdmw.lsdtt_driver(read_prefix = "lg_conception_SRTM30_UTM",
                                 write_prefix= "lg_conception_SRTM30_UTM",
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()

In [ ]:
lsdtt_drive.run_lsdtt_command_line_tool()

## Plot some data

We are now going to do some simple plots using a mapping package that we put together. There are more general ways to visualise data, but this makes pretty pictures quickly.  

In [ ]:
%matplotlib inline
Base_file = "lg_conception_SRTM30_UTM"
DataDirectory = "./"
this_img = lsdmw.SimpleHillshade(DataDirectory,Base_file,cmap="gist_earth", save_fig=False, size_format="geomorphology")

In [ ]:
Base_file = "lg_conception_SRTM30_UTM"
Drape_prefix = "lg_conception_SRTM30_UTM_SLOPE"
DataDirectory = "./"
img_name2 = lsdmw.SimpleDrape(DataDirectory,Base_file, Drape_prefix, 
                              cmap = "bwr", cbar_loc = "right", 
                              cbar_label = "Gradient (m/m)",
                              save_fig=False, size_format="geomorphology",
                              colour_min_max = [0,1.25])

## Get some channel profiles

Okay, we will now run a different analysis. We will get some channel profiles. 

In [ ]:
lsdtt_parameters = {"print_basin_raster" : "true",
                    "print_chi_data_maps" : "true"}
lsdtt_drive = lsdmw.lsdtt_driver(read_prefix = "lg_conception_SRTM30_UTM",
                                 write_prefix= "lg_conception_SRTM30_UTM",
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()

In [ ]:
lsdtt_drive.run_lsdtt_command_line_tool()

Now we are going to read in the channel data using geopandas

In [ ]:
df = pd.read_csv("lg_conception_SRTM30_UTM_chi_data_map.csv")

The csv file has latitude and longitude coordinates, so we can use these to register the geometry with a `geopandas` dataframe, and we can also set the coordinate reference system (`set_crs`). The `latitude` and `longitude` data produced by `lsdtopotools` is always in `epsg=4326`.

In [ ]:
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
gdf = gdf.set_crs(epsg=4326)
print(gdf.head())

So after some experimenting, it turns out that the plotting functions are not very happy if you don't keep everything in the same coordinate system. So I need to transform the geopandas dataframe to the correct UTM coordinate system. You will probably need to make sure your rasters and your dataframes are alyways in the same coordinate system.

We need to know what coordinate system our data is in. Lets find that out.

In [ ]:
DEMdata = rio.open('lg_conception_SRTM30_UTM.bil')
bounds = DEMdata.bounds
DEMdata.crs

DEMHSdata = rio.open('lg_conception_SRTM30_UTM_hs.bil')
arrayHS = DEMHSdata.read(1)
arrayHS[arrayHS==-9999]=np.nan


Okay, now tell geopandas to convert the coordinate system

In [ ]:
gdf2 = gdf.to_crs("EPSG:32610")

Lets have a look at these channels

In [ ]:
Extent = [bounds.left,bounds.right,bounds.bottom,bounds.top]

array = DEMdata.read(1)
array[array<-5]=np.nan

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection=ccrs.UTM(46))
# Limit the extent of the map to a small longitude/latitude range.
#ax.set_extent(Extent, crs=ccrs.UTM(46))
ax.coastlines()
ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
#print('Projecting and plotting image (this may take a while)...')
ax.imshow(array, extent=Extent, transform=ccrs.UTM(46), origin="upper", cmap='terrain',alpha=0.5,zorder=-2)
ax.imshow(arrayHS, extent=Extent, transform=ccrs.UTM(46), origin="upper", cmap='gray',alpha=1,zorder=-3)
gdf2.plot(ax=ax, markersize=1, column='chi', zorder=10,cmap="jet")
fig.savefig("test.png", dpi=800)
fig.show()

Okay, lets plot a channel profile! We first need to select a basin

In [ ]:
gdf_b1 = gdf2[(gdf2['basin_key'] == 0)]
print(gdf_b1.head())

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
plt.scatter(gdf_b1.chi,gdf_b1.elevation,c=gdf_b1.flow_distance,s=1)
plt.xlabel(r"$\chi$ (m)")
plt.ylabel("elevation (m)")
fig.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
plt.scatter(gdf_b1.flow_distance,gdf_b1.elevation,c=gdf_b1.flow_distance,s=1)
plt.xlabel(r"$\chi$ (m)")
plt.ylabel("elevation (m)")
fig.show()